# RabbitMQ example consumer code

In [ ]:
# import the pika library
import pika, logging, time
from pika.exchange_type import ExchangeType

print('pika version: %s' % pika.__version__)

In [ ]:
# a dummy epam processs function
def epam_process_function(msg):
  print(" EPAM processing")
  print(" [x] Received message " + str(msg))

  # delays for 5 seconds
  time.sleep(5) 

  print(" EPAM processing finished!")
  return;

In [ ]:
# callback function to be called on incoming messages
def callback(ch, method, properties, body):
  epam_process_function(body)

In [ ]:
# configure parameters, change the rabbitmq_url according to rabbitmq deployment
rabbitmq_url = 'rabbitmq.rabbitmq.svc.cluster.local'
username = 'pcaidemo'
password = 'Changeme!2345'
usercredentials = pika.PlainCredentials(username, password)

# connect to rabbitmq instance
parameters = pika.ConnectionParameters('rabbitmq.rabbitmq.svc.cluster.local', 5672, '/', usercredentials)
connection = pika.BlockingConnection(parameters)

# start a channel
channel = connection.channel()

# declare an exchange
channel.exchange_declare(exchange='epam-exchange', exchange_type=ExchangeType.direct)

# declare a queue
queue_result = channel.queue_declare(queue='epam-process') 
queue_name = queue_result.method.queue

# set up a subscription on the queue
channel.basic_consume('epam-process', callback, auto_ack=True)

# start consuming
channel.start_consuming()

# close the connection
connection.close()